# Different Solutions for 1-Dimensional-1-Phase Problem

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1VsPyLcLGSQwQD8EHkY-UdUPreWSPvy9Y/view?usp=drive_link)

*Author: Zakariya Abugrin | Date: November 2023*

## Introduction

The solution for a simple 1-dimensional-1-phase (1D1P) problem is available in all solutions which includes: `analytical`, `numerical`, `neurical`. In tutorial, we will see how we can build all solutions and compare them together. 

## Prerequisites:
Before you can follow this tutorial, you need to understand the following tutorials:
1. [Understand scalers Module](/user_guide/tutorials/tutorial_understand_scalers_module.html).
2. [Understand Compiling](/user_guide/tutorials/tutorial_understand_compiling.html).

## Import `reservoirflow`

We start with importing `reservoirflow` as `rf`. The abbreviation `rf` refers to `reservoirflow` where all modules under this library can be accessed. `rf` is also used throughout the [documentation](/api/API.html). We recommend our users to stick with this convention.

In [1]:
import reservoirflow as rf
import numpy as np

print(rf.__version__)

0.1.0


## Create a Model

In [2]:
run_config = dict(
    nsteps=100,
    threading=True,
    vectorize=True,
    check_MB=True,
    print_arrays=False,
    isolver=None,  #'cgs',
)

In [3]:
def create_model(nx=101, n_w=2, comp=False):
    if comp:
        g_comp = 1 * 10**-6
        f_comp = 1 * 10**-5
    else:
        g_comp = None
        f_comp = None

    grid = rf.grids.RegularCartesian(
        nx=nx,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=40,
        phi=0.27,
        kx=270,
        comp=g_comp,
        dtype="double",
    )

    fluid = rf.fluids.SinglePhase(
        mu=0.5,
        B=1,
        comp=f_comp,
        dtype="double",
    )

    model = rf.models.BlackOil(
        grid, fluid, pi=4000, dtype="double", dt=1, verbose=False, sparse=True
    )
    if n_w > 0:
        well_ids = np.cumsum([nx // (n_w + 1) for i in range(n_w)])
        print("Wells_ids:", well_ids)
        for well_id in well_ids:
            # model.set_well(id=well_id, q=-600, s=1.5, r=3.5)
            model.set_well(id=well_id, pwf=1000, s=1.5, r=3.5)
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 4000)})
    else:
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 1000)})
        # model.set_boundaries({0: ("pressure", 1000), nx+1: ("pressure", 4000)})

    return model

In [4]:
model = create_model(n_w=0, comp=True)

In [5]:
model.compile("numerical", "FDM")

[info] FDM was assigned as model.solution.


In [6]:
help(model.run)

Help on method run in module reservoirflow.solutions.numerical.fdm:

run(nsteps=10, threading=True, vectorize=True, check_MB=True, print_arrays=False, isolver=None) method of reservoirflow.solutions.numerical.fdm.FDM instance
    Perform a simulation run for nsteps.
    
    Parameters
    ----------
    nsteps : int, optional
        _description_
    threading : bool, optional
        _description_
    check_MB : bool, optional
        _description_
    isolver : str, optional
        iterative solver for sparse matrices. Available solvers are
        ["bicg", "bicgstab", "cg", "cgs", "gmres", "lgmres",
        "minres", "qmr", "gcrotmk", "tfqmr"].
        If None, direct solver is used. Only relevant when argument
        sparse=True. Direct solver is recommended for more accurate
        calculations. To improve performance, "cgs" is recommended
        to increase performance while option "minres" is not recommended due to
        high MB error. For more information check [1][2].


In [7]:
model.run(**run_config)

[info] Simulation run started: 100 timesteps.


  0%|          | 0/100 [00:00<?, ?steps/s]


ValueError: Property kx is not defined. 

```{include} /_static/comments_section.md
```